<a href="https://colab.research.google.com/github/DenisDeFaria/Challenge_QubeRT/blob/main/Qube_RT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try :
  from google.colab import drive
  from google.colab import files
  drive.mount('/content/drive')
  PATH_GC = '/content/drive/My Drive/Challenge' 
except Exception:
  print("Vous n'êtes pas sur Google Colab, vous êtes en local, assurez-vous d'avoir ajouter le Path du dossier Colab_sources")
  PATH_GC = ''#Emplacement du dossier /Colab_sources

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
X_train = pd.read_csv(PATH_GC+'/X_train.csv', index_col=0, sep=',')
X_train.columns.name = 'date'

Y_train = pd.read_csv(PATH_GC+'/Y_train.csv', index_col=0, sep=',')
Y_train.columns.name = 'date'


In [ ]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


# Function

In [ ]:
your_list = list(np.random.permutation(np.arange(0,50)))
list_train = your_list[0:25]
list_test = your_list[25:50]

In [ ]:
Y_train_data = Y_train.iloc[list_train]
Y_test_data = Y_train.iloc[list_test]

In [ ]:
X_train_data = X_train.iloc[list_train]
X_train_data = pd.concat([ X_train_data.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()

X_test_data = X_train.iloc[list_test]
X_test_data = pd.concat([ X_test_data.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """


In [ ]:
def fitBeta(A, train = True):
    if train:
      X = X_train_data
      Y = Y_train_data
    else:
      X = X_test_data
      Y =  Y_test_data

    X = torch.tensor(X.values)
    predictors = torch.mm(X, A) # the dataframe of the 10 factors created from A with the (date, stock) in index

    targets = Y.T.stack()
    targets = torch.tensor(targets.values)
    targets = targets[:,None]

    beta = torch.linalg.inv(torch.mm(predictors.T, predictors))
    beta = torch.mm(beta, predictors.T)
    beta = torch.mm(beta, targets)

    return beta

def metric(df_y_true, df_y_pred):
    """ Compute metric. """
    if df_y_pred is None:  # If the y_pred has only zeroes, the metric is set to -1.
        return -1.0
    
    y_true = df_y_true.T
    y_pred = df_y_pred.T
    
    y_true = y_true.div(y_true.pow(2.0).sum(1).pow(0.5), 0)
    y_pred = y_pred.div(y_pred.pow(2.0).sum(1).pow(0.5), 0)

    mean_overlap = (y_true * y_pred).sum(1).mean()

    return mean_overlap   

def metric_train(A, beta, train = True): 
    if train:
      X = X_train_data
      Y = Y_train_data
      nbr = 25
    else:
      X = X_test_data
      Y = Y_test_data
      nbr = 25

    X = torch.tensor(X.values)
    beta = beta.reshape(10,1)
    Ypred = torch.mm(X, A)
    Ypred = torch.mm(Ypred, beta)
    Ypred = Ypred.reshape(504, nbr).T 
    Ytrue = torch.tensor(Y.values)
    
    for k in range(504):
        Ytrue[:,k] = Ytrue[:,k].clone()/torch.linalg.norm(Ytrue[:,k].clone())

        Ypred[:,k] = Ypred[:,k].clone()/torch.linalg.norm(Ypred[:,k].clone())

    
    meanOverlap = torch.mean(torch.sum(Ytrue*Ypred,dim =  0))

    return 0.2-meanOverlap





# Function Submission

In [ ]:
def transform_submission_to_ypred(df_A,df_beta, x_test, y_test):
    """ Transform submission output (A, beta) into predicted returns S_t."""
    df_A = df_A.to_numpy()
    df_beta = df_beta.to_numpy()
    
    A = df_A.reshape((250, 10))
    beta = df_beta.reshape(10)

    E = pd.DataFrame(A.T @ A - np.eye(10)).abs()  

    # check orthogonality of A
    if any(E.unstack() > 1e-6): 
        return None

    x_test = x_test.T
    y_test = y_test.T

    x_test = x_test[y_test.columns]

    x_test_reshape = pd.concat([x_test.shift(i+1).stack(dropna=False) for i in range(250)], 1).dropna()
    y_pred = (x_test_reshape @ A @ beta).unstack()

    return y_pred.T

def parametersTransform(A, beta, D=250, F=10):
    
    if A.shape != (D, F):
        print('A has not the good shape')
        return
    
    if beta.shape[0] != F:
        print('beta has not the good shape')
        return        
    
    output = np.hstack( (np.hstack([A.T, beta.reshape((F, 1))])).T )
    
    return output

def fitBeta_df(A):
    predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
    targets = Y_train.T.stack()
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

# Training

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## First model

In [ ]:
class descente1():
    def __init__(self):
        self.A = torch.rand((250,10), requires_grad = True)

    
    def parameters(self):
      return([self.A])

    def forward(self):
        return torch.linalg.qr(self.A, mode='reduced')[0]

In [ ]:
n_epochs = 100

In [ ]:
min_loss = 0.2

In [ ]:
for retest in range(1000):

  model = descente1()
  optimizer = torch.optim.Adam(model.parameters(), lr= 0.05)
  total_loss = 0
  Loss_mat = []
  Loss_test = []
  for j in range(n_epochs):
    
    Mat = model.forward().double()
    Mat.to(device)

    #beta  = Mat.clone().detach()
    #beta.to(device)
    #beta =  fitBeta(beta, train = True)
    beta = torch.ones(10).double()

    #loss = metric_train(Mat, beta  )
    loss =  metric_train(Mat, beta, train= True)



    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    loss_test = metric_train(Mat.detach() , beta.detach(), train = False)

    if loss_test < min_loss:
      min_loss = loss_test
      Mat_fin = Mat.clone().detach().numpy()
      beta_fin = beta.clone().detach().numpy()

    if j >0:
      Loss_test.append(loss_test)
      Loss_mat.append(loss.item())

    if j%10 ==0 and  j>0:
      abscisse = np.linspace(1,len(Loss_mat), len(Loss_mat))
      plt.plot(abscisse, Loss_mat)
      plt.plot(abscisse, Loss_test)
      plt.show()
      print('At epochs '+str(j+1)+' : '+str(total_loss/(j+1)))
      print('min_loss : '+str(min_loss))
    #Mat_fin = pd.DataFrame(Mat_fin)
    #Mat_fin.to_csv(PATH_GC+'/Challenge/random'+str(j))

In [ ]:
0.2-0.1321

0.06790000000000002

# Second model

In [ ]:
class descente():
    def __init__(self):
        super(descente, self).__init__()
        #self.A = torch.tensor(A_QRT, requires_grad = True)
        self.A = torch.rand((250,10), requires_grad = True)
        self.B = torch.ones((10), requires_grad = True)

    def parameters(self):
      return([self.A, self.B])

    def forward(self):
        return torch.linalg.qr(self.A, mode='reduced')[0], self.B

In [ ]:
min_loss = 0.2

In [ ]:
n_epochs = 60

In [ ]:
for retest in range(1000):

  model = descente()
  optimizer = torch.optim.Adam(model.parameters(), lr= 0.05)
  total_loss = 0
  Loss_mat = []
  Loss_test = []
  for j in range(n_epochs):
    
    Mat, beta = model.forward()
    
    Mat.to(device)
    Mat = Mat.double()

    beta.to(device)
    beta = beta.double()
    


    loss =  metric_train(Mat, beta, train = True)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    loss_test = metric_train(Mat.detach() , beta.detach(), train = False)

    if loss_test < min_loss:
      min_loss = loss_test
      Mat_fin = Mat.clone().detach().numpy()
      beta_fin = beta.clone().detach().numpy()

    if j >0:
      Loss_test.append(loss_test)
      Loss_mat.append(loss.item())

    if j%10 ==0 and  j>0:
      abscisse = np.linspace(1,len(Loss_mat), len(Loss_mat))
      plt.plot(abscisse, Loss_mat)
      plt.plot(abscisse, Loss_test)
      plt.show()
      print('At epochs '+str(j+1)+' : '+str(total_loss/(j+1)))
      print('min_loss : '+str(min_loss))
  #    Mat_fin = pd.DataFrame(Mat_fin)
  #    Mat_fin.to_csv('./Challenge/A'+str(j))

  #    beta_fin = pd.DataFrame(beta_fin)
  #    beta_fin.to_csv('./Challenge/beta'+str(j))
      

In [ ]:
print(j)

Mat_df = pd.DataFrame(Mat_fin)
Mat_df.to_csv(PATH_GC+'/Challenge/A'+str(j))

beta_df = pd.DataFrame(beta_fin)
beta_df.to_csv(PATH_GC+'/Challenge/beta'+str(j))

77


In [ ]:
df_A = pd.DataFrame(Mat_fin)
df_beta = pd.DataFrame(beta_fin)

y_pred = transform_submission_to_ypred(df_A,df_beta, X_train, Y_train)
result = metric(Y_train, y_pred)
print(result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


0.1088998129069301


In [ ]:
PATH_GC+'/Challenge/A'+str(j)


'/content/drive/My Drive/Challenge/A29'

# Submission 

In [ ]:


pathA = '/content/drive/My Drive/Challenge/A' + str(j)
pathB = '/content/drive/My Drive/Challenge/beta' + str(j)
pathOutput = str(j)

A = pd.read_csv( pathA, index_col=0, sep=',').to_numpy()
beta = pd.read_csv(pathB, index_col=0, sep=',').to_numpy()

output = parametersTransform(A, beta)
pd.DataFrame(output).to_csv(PATH_GC+'/Challenge/'+pathOutput+'.csv')

# Genetic

In [ ]:
def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool


def metric_train_df(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X_train_reshape @ A @ beta).unstack().T         
    Ytrue = Y_train
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap



def fitBeta_df(A):
    
    predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
    targets = Y_train.T.stack()
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

In [ ]:
import random
import numpy as np

In [ ]:
class Individual(object):

  def __init__(self, chromosome):
        self.chromosome = chromosome 
        self.fitness = self.cal_fitness()


  @classmethod
  def mutated_genes(self):
    """
    Create random genes for mutation
    """

    gene = np.random.randn(250)
    return gene

  @classmethod
  def create_gnome(self):
    '''
    Create chromosome or string of genes
    '''
    gnome = np.random.randn(250, 10)

    return np.linalg.qr(gnome)[0]


  def mate(self, par2):
    """
    Perform the mutation
    """

    child_chromosome = np.zeros((250,10))

    for column in range(10):

      prob = random.random()

      if prob < 0.45:
        child_chromosome[:, column] = self.chromosome[:, column]
      
      elif prob >= 0.45 and prob < 0.90:
        child_chromosome[:, column] = par2.chromosome[:, column]

      else:
        child_chromosome[:, column] = self.mutated_genes()

    return Individual(np.linalg.qr(child_chromosome)[0])


  def cal_fitness(self):
    """
    Calculate metric
    """
    beta = fitBeta_df(self.chromosome)

    fitness = metric_train_df(self.chromosome , beta)

    return fitness






In [ ]:
population = []
max = 0
nbr_of_iteration = 1000
population_size = 250

for _ in range(population_size):
  gnome = Individual.create_gnome()

  population.append(Individual(gnome))

for generation in range(nbr_of_iteration):
  population = sorted(population, key = lambda x:x.fitness, reverse = True)

  if population[0].fitness > max:

    best_chromosome = population[0].chromosome
    max = population[0].fitness
    print(f'Best metric found is for generation : {generation} and the value is {max}' )

  new_generation = []

  s = int((10 * population_size)/100)

  new_generation.extend(population[:s])

  s = int((90 * population_size)/100)


  for _ in range(s):
    parent1 = random.choice(population[:50])
    parent2 = random.choice(population[:50])

    child = parent1.mate( parent2)

    new_generation.append(child)

  population = new_generation

  #print("Generation: {}\tFitness: {}".format(generation, population[0].fitness))


Best metric found is for generation : 0 and the value is 0.04637718234826114
Best metric found is for generation : 1 and the value is 0.049888626383343855
Best metric found is for generation : 2 and the value is 0.05561015883209621
Best metric found is for generation : 3 and the value is 0.0635894568800699
Best metric found is for generation : 4 and the value is 0.06368096618086531
Best metric found is for generation : 5 and the value is 0.06736606642146517
Best metric found is for generation : 6 and the value is 0.07174376696529719
Best metric found is for generation : 7 and the value is 0.0739520582039841
Best metric found is for generation : 8 and the value is 0.07537385416842961
Best metric found is for generation : 9 and the value is 0.0779023671340428
Best metric found is for generation : 10 and the value is 0.07809450197805123
Best metric found is for generation : 12 and the value is 0.07929543341294229
Best metric found is for generation : 13 and the value is 0.0814506545868405

KeyboardInterrupt: ignored

In [ ]:
transform_submission_to_ypred(pd.DataFrame(best_chromosome), pd.DataFrame(fitBeta_df(best_chromosome)), X_train, Y_train)

In [ ]:
output = parametersTransform(best_chromosome,  fitBeta_df(best_chromosome))

In [ ]:
pathOutput ="generic_algo_1"

pd.DataFrame(output).to_csv(PATH_GC+'/'+pathOutput+'.csv')